**BÀI THỰC HÀNH 5 - PHÂN LỚP VĂN BẢN VỚI BERT**

NỘI DUNG:

*   CHUẨN BỊ NƠI LƯU TRỮ VÀ DỮ LIỆU
*   SỬ DỤNG THƯ VIỆN


**1) CHUẨN BỊ NƠI LƯU TRỮ VÀ DỮ LIỆU**

Chúng ta cần upgrade kagglehub và scikit-learn để sử dụng phiên bản mới nhất.

In [ ]:
!pip install --upgrade kagglehub &>> /dev/null
!pip install transformers &>> /dev/null
!pip install --upgrade scikit-learn &>> /dev/null

Tiếp theo, chúng ta sẽ khai báo sử dụng các package cần thiết cho dữ liệu, load BERT pretrained model và huấn luyện BERT.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import torch
import numpy as np
from torch.utils import data
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

import seaborn as sns #Charts

import os


Tiếp theo, chúng ta sẽ download dữ liệu SMS Spam từ kaggle. Và lưu file spam.csv vào thư mục /Content/

In [ ]:
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")
os.system("cp \"{}/spam.csv\" /content/spam.csv".format(path))


100%|██████████| 211k/211k [00:00<00:00, 404kB/s]

Extracting files...


0

**2) SỬ DỤNG THƯ VIỆN**

Chúng ta sẽ sử dụng lớp AutoModelForSequenceClassification để lưu trữ và cập nhật bộ tham số của mô hình phân lớp văn bản sử dụng BERT pretrained model, lớp Trainer để thực hiện việc train mô hình.

In [ ]:
def preprocess(msg, cls, tokenizer, clslookup):
  results = []
  for i in range(len(msg)):
    inp = tokenizer(msg[i], truncation=True)
    results.append({"id":i, "input_ids": inp["input_ids"], "labels": clslookup[cls[i]]})
  return results

dataset = pd.read_csv("spam.csv", encoding="windows-1252")
dataset.info()
dataset = dataset.drop(dataset[["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"]], axis=1)
dataset.rename(columns = {"v1":"Class", "v2":"Message"}, inplace = True)
dataset.head()

classes = list(dataset["Class"])
messages = list(dataset["Message"])

clsset = set(classes)

clslookup = {cls:idx for idx, cls in enumerate(clsset)}
idxlookup = {idx:cls for idx, cls in enumerate(clsset)}

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased", num_labels=len(idxlookup), id2label=idxlookup, label2id=clslookup
)

samples = preprocess(messages, classes, tokenizer, clslookup)

train, test = train_test_split(samples, test_size=0.2, random_state=42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(train[0])

{'id': 1978, 'input_ids': [101, 2053, 1045, 1005, 1049, 1999, 1996, 2168, 4049, 1012, 2145, 2182, 2012, 2026, 3566, 2015, 1012, 4638, 2033, 2041, 2006, 10930, 1012, 1045, 1005, 1049, 2431, 6248, 1012, 102], 'labels': 1}


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    #output_dir="thư mục lưu model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    #eval_strategy="epoch",
    save_strategy="epoch",
    #load_best_model_at_end=True,
    #push_to_hub=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    #eval_dataset=test,
    processing_class=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
500,0.052600


TrainOutput(global_step=558, training_loss=0.050067566201678314, metrics={'train_runtime': 133.0998, 'train_samples_per_second': 66.972, 'train_steps_per_second': 4.192, 'total_flos': 289778799709320.0, 'train_loss': 0.050067566201678314, 'epoch': 2.0})

Sử dụng mô hình để dự đoán

In [ ]:
tagger = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
sent = "You should probably train this model on a down-stream task to be able to use it"
tagger(sent)

[{'label': 'ham', 'score': 0.9928953051567078}]